In [1]:
# import all necessary packages for webscraping a list of Bundesliga first devision teams

import time
import requests
from bs4 import BeautifulSoup
import re # support regular expressions
import pandas as pd

In [3]:
# get content of the page that lists all current Bundeliga teams on tm.de

page = requests.get("https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1")
html = page.content

In [4]:
# parse the html document into type BeautifulSoup

bs = BeautifulSoup(html, 'html.parser')

In [5]:
# display the content using prettifier method

print(bs.prettify())

<html>
 <head>
  <title>
   404 Not Found
  </title>
 </head>
 <body bgcolor="white">
  <center>
   <h1>
    404 Not Found
   </h1>
  </center>
  <hr/>
  <center>
   nginx
  </center>
 </body>
</html>



In [7]:
# find children properties using bs attributes

list(bs.findChildren())

[<html>
 <head><title>404 Not Found</title></head>
 <body bgcolor="white">
 <center><h1>404 Not Found</h1></center>
 <hr/><center>nginx</center>
 </body>
 </html>,
 <head><title>404 Not Found</title></head>,
 <title>404 Not Found</title>,
 <body bgcolor="white">
 <center><h1>404 Not Found</h1></center>
 <hr/><center>nginx</center>
 </body>,
 <center><h1>404 Not Found</h1></center>,
 <h1>404 Not Found</h1>,
 <hr/>,
 <center>nginx</center>]

In [8]:
bs.find('head')

<head><title>404 Not Found</title></head>

In [9]:
bs.find('h1')

<h1>404 Not Found</h1>

In [10]:
# since the tag 'h1' is getting used numerous times another method fits the requirement even better

bs.find_all('h1')

[<h1>404 Not Found</h1>]

In [11]:
titles = bs.find_all(['h1', 'h2','h3','h4','h5','h6'])
print([title for title in titles])

[<h1>404 Not Found</h1>]


In [12]:
bs.find('title').get_text()

'404 Not Found'